In [1]:
import gensim
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import numpy as np
import re

import string
import nltk

In [2]:
pathToDatasets = '../datasets/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
word_vectors = api.load("glove-wiki-gigaword-100")

In [3]:
# positiveWordList = ['affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative']
# negativeWordList = ['varice', 'avaricious', 'avariciously', 'avenge', 'averse', 'aversion', 'aweful', 'awful', 'awfully', 'awfulness']

# dimensionsDictionary = {}
# countOfWords = 0;

# def countDimensions(listOfDimensions):
#   for dimension in listOfDimensions:
#     if (dimensionsDictionary.get(dimension) == None):
#       dimensionsDictionary[dimension] = 1;
#     else:
#       dimensionsDictionary[dimension] +=1;


In [4]:
# def sortedDictionary(dictUnsorted):
#   sorted_d = sorted(dictUnsorted.items(), key=lambda x: x[1], reverse=True)
#   return sorted_d;



In [5]:
# vocabulary = word_vectors.vocab;
# for posWord in positiveWordList:
#   if(posWord not in vocabulary):
#     continue  
#   posVector = word_vectors.word_vec(posWord)
#   for negWord in negativeWordList:
#     if(negWord not in vocabulary):
#       continue
#     negVector = word_vectors.word_vec(negWord)
#     diffBetween = posVector - negVector
#     sortedList = sorted(range(len(diffBetween)), key=lambda k: diffBetween[k], reverse=True)
#     similarityScore = word_vectors.similarity(posWord, negWord);
# #     print("Dimensions for word {0} and word {1} is {2} with similarity scores: {3}".format(posWord, negWord, sortedList[0:3], similarityScore))
#     countDimensions(sortedList[0:3]);
#     countOfWords+=1;
#     # print("----- {} -----".format(diffBetween));

# # # print(dimensionsDictionary);
# print(sortedDictionary(dimensionsDictionary));
# print("Count of number of word pairs : {0}".format(countOfWords))



In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize



senty = SentimentIntensityAnalyzer()


/cm/shared/languages/Anaconda3-5.2.0/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [7]:
def listReplacements(word):
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=5)]
    
    return possibleReplacements

def cleanAndTokenizeText(text):
    text = text.lower()
    newString = ""
    for char in text:
        if char not in string.punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

In [22]:
# userInput = input()
userInput = "This is the best sentence i have ever heard!"

increasePositive = True
mainSentiment = senty.polarity_scores(userInput)['compound']

userInputTokens = cleanAndTokenizeText(userInput)
print("Original Sentence :# {0} : {1} #".format(userInput,mainSentiment))
print("Alternatives: ")
for ind,word in enumerate(userInputTokens):
    score = senty.polarity_scores(word)['compound']
    newUserTokens = userInputTokens[:];
    if(score != 0.0 ):
        replacements = listReplacements(word)        
       
        for newWord in replacements:
            newUserTokens[ind] = newWord;
            newString = ' '.join(newUserTokens)
            sentimentOfNewString = senty.polarity_scores(newString)['compound']
            if(sentimentOfNewString >= mainSentiment):
                print("{0} : {1} ++POS++".format(newString,sentimentOfNewString))
            else:
                print("{0} : {1} --NEG--".format(newString,sentimentOfNewString))
        newUserTokens = userInputTokens[:];


Original Sentence :# This is the best sentence i have ever heard! : 0.6996 #
Alternatives: 
this is the good sentence i have ever heard : 0.4939 --NEG--
this is the well sentence i have ever heard : 0.34 --NEG--
this is the better sentence i have ever heard : 0.4939 --NEG--
this is the winning sentence i have ever heard : 0.5719 --NEG--
this is the one sentence i have ever heard : 0.0772 --NEG--
this is the best sentences i have ever heard : 0.6597 --NEG--
this is the best imprisonment i have ever heard : 0.6369 --NEG--
this is the best jail i have ever heard : 0.6369 --NEG--
this is the best sentencing i have ever heard : 0.5574 --NEG--
this is the best conviction i have ever heard : 0.6369 --NEG--
